<h2 style="color:tomato;">Task 4: Nested Set Model Implementation</h2>
Write a Python program to manipulate the Nested Set Model: <br><br>

●	Create Nested Set Model: Develop a function or class to convert hierarchical data into the nested set model.<br><br>
●	Retrieve Parent-Child Relationships: Calculate and organize parent-child relationships from the nested set model.<br><br>
●	Measure Performance: Optimize and evaluate the program's performance when dealing with nested sets on larger datasets.


<h1 style="color:tomato;">Solution:</h1>

I write a python program work with hierarchical data Employee(id:integer) and their suppervisors example.<br> <br>
Here is my hierarchical data:<br> <br>
<image src="nestedsetEx.png" alt="imgage">


Then i used the library in Python "sqlalchemy_mptt" (document link: <a>https://pypi.org/project/sqlalchemy_mptt/</a> ) to demonstrate my example.

In [43]:

from sqlalchemy import (create_engine, Column, Integer, String, select, case,
    func)
from sqlalchemy.orm import Session, aliased
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import event

Base = declarative_base()

class Employee(Base):
    __tablename__ = 'personnel'
    __mapper_args__ = {
        'batch': False  # allows extension to fire for each
                        # instance before going to the next.
    }

    parent = None

    id = Column(Integer, primary_key=True)

    left = Column("lft", Integer, nullable=False)
    right = Column("rgt", Integer, nullable=False)

    def __repr__(self):
        return "Employee(%d, %d, %d)" % (self.id, self.left, self.right)

@event.listens_for(Employee, "before_insert")
def before_insert(mapper, connection, instance):
    if not instance.parent:
        instance.left = 1
        instance.right = 2
    else:
        personnel = mapper.mapped_table
        right_most_sibling = connection.scalar(
            select([personnel.c.rgt]).
                where(personnel.c.id == instance.parent.id)
        )

        connection.execute(
            personnel.update(
                personnel.c.rgt >= right_most_sibling).values(
                    lft=case(
                        [(personnel.c.lft > right_most_sibling,
                            personnel.c.lft + 2)],
                        else_=personnel.c.lft
                    ),
                    rgt=case(
                        [(personnel.c.rgt >= right_most_sibling,
                                personnel.c.rgt + 2)],
                            else_=personnel.c.rgt
                      )
            )
        )
        instance.left = right_most_sibling
        instance.right = right_most_sibling + 1


engine = create_engine('sqlite://', echo=True)

Base.metadata.create_all(engine)

session = Session(bind=engine)

node1 = Employee(id=1)
node2 = Employee(id=2)
node4 = Employee(id=4)
node7 = Employee(id=7)
node3 = Employee(id=3)
node5 = Employee(id=5)
node6 = Employee(id=6)
node8 = Employee(id=8)
node10 = Employee(id=10)
node9 = Employee(id=9)
node11 = Employee(id=11)


node2.parent=node1
node4.parent=node1
node7.parent=node1
node3.parent=node2
node5.parent=node4
node6.parent=node4
node8.parent=node7
node10.parent=node7
node9.parent=node8
node11.parent=node10

# # the order of "add" is important here.  elements must be added in
# # the order in which they should be INSERTed.
session.add_all([node1, node2, node4, node7, node3, node5, node6 , node8 , node10 , node9, node11])
session.commit()
print(node1)
print(session.query(Employee).all())

# 1. Find an employee and all their supervisors, no matter how deep the tree.
ealias = aliased(Employee)
print(session.query(Employee).\
            filter(ealias.left.between(Employee.left, Employee.right)).\
            filter(ealias.id == 9).all())

#2. Find the employee and all their subordinates.
# (This query has a nice symmetry with the first query.)
print(session.query(Employee).\
    filter(Employee.left.between(ealias.left, ealias.right)).\
    filter(ealias.id == 7).all())

#3. Find the level of each node, can also print the tree
# as an indented listing.
for indentation, employee in session.query(
            func.count(Employee.id).label('indentation') - 1, ealias).\
    filter(ealias.left.between(Employee.left, Employee.right)).\
    group_by(ealias.id).\
        order_by(ealias.left):
    print("    " * indentation + str(employee))



2023-12-27 16:07:57,414 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-27 16:07:57,415 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("personnel")
2023-12-27 16:07:57,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-27 16:07:57,417 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("personnel")
2023-12-27 16:07:57,417 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-27 16:07:57,419 INFO sqlalchemy.engine.Engine 
CREATE TABLE personnel (
	id INTEGER NOT NULL, 
	lft INTEGER NOT NULL, 
	rgt INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2023-12-27 16:07:57,420 INFO sqlalchemy.engine.Engine [no key 0.00053s] ()
2023-12-27 16:07:57,420 INFO sqlalchemy.engine.Engine COMMIT
2023-12-27 16:07:57,424 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-27 16:07:57,426 INFO sqlalchemy.engine.Engine INSERT INTO personnel (id, lft, rgt) VALUES (?, ?, ?)
2023-12-27 16:07:57,426 INFO sqlalchemy.engine.Engine [generated in 0.00052s] (1, 1, 2)
2023-12-27 16:07:57,428 INFO sqlalchemy

C:\Users\Admin\AppData\Local\Temp\ipykernel_23588\3008336467.py:32: SADeprecationWarning: Use .persist_selectable (deprecated since: 1.3)
  personnel = mapper.mapped_table
Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\sqlalchemy\pool\base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\sqlalchemy\pool\base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\sqlalchemy\engine\default.py", line 683, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 7700 and this is thread id 10076.
Exception closing connection <sqlite3.Connection object at 0x0000024B19B1ED40>
Traceback (most recent call l

2023-12-27 16:07:57,448 INFO sqlalchemy.engine.Engine [cached since 0.01965s ago] (4,)
2023-12-27 16:07:57,451 INFO sqlalchemy.engine.Engine UPDATE personnel SET lft=CASE WHEN (personnel.lft > ?) THEN personnel.lft + ? ELSE personnel.lft END, rgt=CASE WHEN (personnel.rgt >= ?) THEN personnel.rgt + ? ELSE personnel.rgt END WHERE personnel.rgt >= ?
2023-12-27 16:07:57,452 INFO sqlalchemy.engine.Engine [cached since 0.02094s ago] (7, 2, 7, 2, 7)
2023-12-27 16:07:57,452 INFO sqlalchemy.engine.Engine INSERT INTO personnel (id, lft, rgt) VALUES (?, ?, ?)
2023-12-27 16:07:57,452 INFO sqlalchemy.engine.Engine [cached since 0.02644s ago] (5, 7, 8)
2023-12-27 16:07:57,453 INFO sqlalchemy.engine.Engine SELECT personnel.rgt 
FROM personnel 
WHERE personnel.id = ?
2023-12-27 16:07:57,454 INFO sqlalchemy.engine.Engine [cached since 0.02513s ago] (4,)
2023-12-27 16:07:57,454 INFO sqlalchemy.engine.Engine UPDATE personnel SET lft=CASE WHEN (personnel.lft > ?) THEN personnel.lft + ? ELSE personnel.lft 

<h2 style="color:tomato;">Note:</h2>
To optimize performance on larger datasets, I need to research some techniques like indexing, caching, or data partitioning, especially when dealing with a significant number of nodes in a hierarchical structure. <br><br>
I think this algorithm should be applied to data that is written one (few) times and read many times.